In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile8 = pd.read_pickle("DDD_Train_Decile8.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile8.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile8, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 11s 5ms/step - loss: 0.3823 - acc: 0.8454 - val_loss: 0.3201 - val_acc: 0.8854
Epoch 36/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3953 - acc: 0.8366 - val_loss: 0.3182 - val_acc: 0.8889
Epoch 37/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3748 - acc: 0.8516 - val_loss: 0.3164 - val_acc: 0.8889
Epoch 38/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3813 - acc: 0.8467 - val_loss: 0.3154 - val_acc: 0.8889
Epoch 39/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3756 - acc: 0.8401 - val_loss: 0.3132 - val_acc: 0.8889
Epoch 40/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3852 - acc: 0.8410 - val_loss: 0.3123 - val_acc: 0.8889
Epoch 41/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3781 - acc: 0.8392 - val_loss: 0.3129 - val_acc: 0.8924
Epoch 42/75
2264/2264 [==============================] - 11s

2264/2264 [==============================] - 12s 5ms/step - loss: 0.3218 - acc: 0.8600 - val_loss: 0.2718 - val_acc: 0.8854
Epoch 20/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3188 - acc: 0.8591 - val_loss: 0.2716 - val_acc: 0.8854
Epoch 21/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3173 - acc: 0.8538 - val_loss: 0.2725 - val_acc: 0.8836
Epoch 22/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3237 - acc: 0.8595 - val_loss: 0.2725 - val_acc: 0.8836
Epoch 23/75
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3220 - acc: 0.8600 - val_loss: 0.2719 - val_acc: 0.8836

Epoch 00023: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 24/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3213 - acc: 0.8600 - val_loss: 0.2718 - val_acc: 0.8836
Epoch 25/75
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3159 - acc: 0.8635 - val_loss: 0.2

Epoch 3/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.4593 - acc: 0.8110 - val_loss: 0.3808 - val_acc: 0.8536
Epoch 4/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.4220 - acc: 0.8322 - val_loss: 0.3519 - val_acc: 0.8765
Epoch 5/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.4052 - acc: 0.8304 - val_loss: 0.3422 - val_acc: 0.8818
Epoch 6/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3978 - acc: 0.8401 - val_loss: 0.3278 - val_acc: 0.8801
Epoch 7/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3896 - acc: 0.8383 - val_loss: 0.3175 - val_acc: 0.8818
Epoch 8/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3763 - acc: 0.8405 - val_loss: 0.3002 - val_acc: 0.8871
Epoch 9/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3631 - acc: 0.8503 - val_loss: 0.2951 - val_acc: 0.8907
Epoch 10/100
2264/2264 [========================

Epoch 75/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3192 - acc: 0.8569 - val_loss: 0.2691 - val_acc: 0.8854
Epoch 76/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3220 - acc: 0.8560 - val_loss: 0.2693 - val_acc: 0.8854
Epoch 77/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3266 - acc: 0.8578 - val_loss: 0.2692 - val_acc: 0.8836
Epoch 78/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3219 - acc: 0.8613 - val_loss: 0.2693 - val_acc: 0.8836
Epoch 79/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3201 - acc: 0.8648 - val_loss: 0.2691 - val_acc: 0.8836
Epoch 80/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3227 - acc: 0.8595 - val_loss: 0.2692 - val_acc: 0.8836
Epoch 81/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3197 - acc: 0.8591 - val_loss: 0.2691 - val_acc: 0.8854
Epoch 82/100
2264/2264 [=================

...

2264/2264 [==============================] - 11s 5ms/step - loss: 0.3108 - acc: 0.8591 - val_loss: 0.2666 - val_acc: 0.8871
Epoch 43/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3138 - acc: 0.8604 - val_loss: 0.2664 - val_acc: 0.8871
Epoch 44/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3084 - acc: 0.8613 - val_loss: 0.2664 - val_acc: 0.8871
Epoch 45/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3178 - acc: 0.8578 - val_loss: 0.2664 - val_acc: 0.8854
Epoch 46/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3090 - acc: 0.8626 - val_loss: 0.2664 - val_acc: 0.8854
Epoch 47/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3169 - acc: 0.8582 - val_loss: 0.2663 - val_acc: 0.8871
Epoch 48/50
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3162 - acc: 0.8604 - val_loss: 0.2663 - val_acc: 0.8871
Epoch 49/50
2264/2264 [==============================] - 11s

2264/2264 [==============================] - 11s 5ms/step - loss: 0.3744 - acc: 0.8436 - val_loss: 0.3160 - val_acc: 0.8871
Epoch 52/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3814 - acc: 0.8383 - val_loss: 0.3158 - val_acc: 0.8871
Epoch 53/125
2264/2264 [==============================] - 12s 6ms/step - loss: 0.3762 - acc: 0.8383 - val_loss: 0.3140 - val_acc: 0.8907
Epoch 54/125
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3761 - acc: 0.8397 - val_loss: 0.3130 - val_acc: 0.8907
Epoch 55/125
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3632 - acc: 0.8529 - val_loss: 0.3115 - val_acc: 0.8907
Epoch 56/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3798 - acc: 0.8401 - val_loss: 0.3117 - val_acc: 0.8907
Epoch 57/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3687 - acc: 0.8467 - val_loss: 0.3116 - val_acc: 0.8924
Epoch 58/125
2264/2264 [==============================

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3334 - acc: 0.8560 - val_loss: 0.2766 - val_acc: 0.8801
Epoch 111/125
2264/2264 [==============================] - 14s 6ms/step - loss: 0.3279 - acc: 0.8538 - val_loss: 0.2764 - val_acc: 0.8783
Epoch 112/125
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3301 - acc: 0.8582 - val_loss: 0.2763 - val_acc: 0.8783
Epoch 113/125
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3273 - acc: 0.8600 - val_loss: 0.2760 - val_acc: 0.8783
Epoch 114/125
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3273 - acc: 0.8587 - val_loss: 0.2759 - val_acc: 0.8783
Epoch 115/125
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3219 - acc: 0.8613 - val_loss: 0.2756 - val_acc: 0.8801
Epoch 116/125
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3857 - acc: 0.8432 - val_loss: 0.3141 - val_acc: 0.8924
Epoch 6/100
2264/2264 [==========================

2264/2264 [==============================] - 12s 5ms/step - loss: 0.3283 - acc: 0.8538 - val_loss: 0.2733 - val_acc: 0.8854
Epoch 58/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3260 - acc: 0.8573 - val_loss: 0.2732 - val_acc: 0.8871
Epoch 59/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3174 - acc: 0.8613 - val_loss: 0.2732 - val_acc: 0.8871
Epoch 60/100
2264/2264 [==============================] - 11s 5ms/step - loss: 0.3222 - acc: 0.8591 - val_loss: 0.2732 - val_acc: 0.8871
Epoch 61/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3147 - acc: 0.8626 - val_loss: 0.2731 - val_acc: 0.8889
Epoch 62/100
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3255 - acc: 0.8560 - val_loss: 0.2732 - val_acc: 0.8889
Epoch 63/100
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3211 - acc: 0.8564 - val_loss: 0.2729 - val_acc: 0.8889
Epoch 64/100
2264/2264 [==============================

2264/2264 [==============================] - 12s 5ms/step - loss: 0.3487 - acc: 0.8472 - val_loss: 0.2901 - val_acc: 0.8836
Epoch 17/50
2264/2264 [==============================] - 12s 5ms/step - loss: 0.3414 - acc: 0.8525 - val_loss: 0.2844 - val_acc: 0.8748
Epoch 18/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3459 - acc: 0.8485 - val_loss: 0.2836 - val_acc: 0.8818
Epoch 19/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3392 - acc: 0.8582 - val_loss: 0.2860 - val_acc: 0.8871
Epoch 20/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3378 - acc: 0.8573 - val_loss: 0.2815 - val_acc: 0.8836
Epoch 21/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3331 - acc: 0.8587 - val_loss: 0.2855 - val_acc: 0.8801
Epoch 22/50
2264/2264 [==============================] - 13s 6ms/step - loss: 0.3314 - acc: 0.8600 - val_loss: 0.2874 - val_acc: 0.8765
Epoch 23/50
2264/2264 [==============================] - 12s

In [ ]:
0.86434